In [1]:
# Starters
import pandas as pd
from pathlib import Path

In [2]:
# File Pathing
health_spending_csv = Path("Resources/Health_Care_Expenditures_per_Capita_by State_of_Residence_raw_data.csv")
#health_spending_csv = Path("Resources/HC_Expenditures_per_ State_Clean.csv")

#pd.read
spending_data = pd.read_csv(health_spending_csv,skiprows=[0,1])
#spending_data = pd.read_csv(health_spending_csv,)
spending_data = spending_data.dropna(axis=0,how='any',subset=["Health Spending per Capita"])
spending_data.head()

,Location,Health Spending per Capita
0,United States,"$9,671"
1,Alabama,"$8,741"
2,Alaska,"$13,226"
3,Arizona,"$8,145"
4,Arkansas,"$8,853"


In [3]:
## Spending Data

In [4]:
# File Pathing
health_coverage_csv = Path("Resources/Health_Insurance_Coverage_of_the_Total_Population_raw_data.csv")
#health_coverage_csv = Path("Resources/HI_Population_Coverage_Clean.csv")

#pd.read
coverage_data = pd.read_csv(health_coverage_csv,skiprows=[0,1])
#coverage_data = pd.read_csv(health_coverage_csv)
coverage_data = coverage_data.dropna(axis=0,how='any',subset=["Employer"])
coverage_data.head()

,Location,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total,Footnotes
0,United States,0.496,0.059,0.198,0.142,0.014,0.092,1.0,1.0
1,Alabama,0.472,0.055,0.195,0.160,0.021,0.097,1.0,NaN
2,Alaska,0.484,0.035,0.213,0.100,0.053,0.115,1.0,NaN
3,Arizona,0.451,0.052,0.210,0.161,0.015,0.111,1.0,NaN
4,Arkansas,0.420,0.054,0.262,0.159,0.014,0.091,1.0,NaN


In [5]:
# Merging
merged_per_state = pd.merge( spending_data, coverage_data, how="left", on=["Location", "Location"] )
merged_per_state.head()

,Location,Health Spending per Capita,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total,Footnotes
0,United States,"$9,671",0.496,0.059,0.198,0.142,0.014,0.092,1.0,1.0
1,Alabama,"$8,741",0.472,0.055,0.195,0.160,0.021,0.097,1.0,NaN
2,Alaska,"$13,226",0.484,0.035,0.213,0.100,0.053,0.115,1.0,NaN
3,Arizona,"$8,145",0.451,0.052,0.210,0.161,0.015,0.111,1.0,NaN
4,Arkansas,"$8,853",0.420,0.054,0.262,0.159,0.014,0.091,1.0,NaN


In [6]:
# File Pathing
#health_coverage_csv = Path("Resources/Health_Insurance_Coverage_of_the_Total_Population_raw_data.csv")
hd_mortality_csv = Path("Resources/Heart_Disease_Mortality_by_State_data-table.csv")

#pd.read
mortaility_data = pd.read_csv(hd_mortality_csv)
mortaility_data = mortaility_data.loc[(mortaility_data["YEAR"] == 2019)]
mortaility_data = mortaility_data.reset_index(drop=True)
XX_to_full = {"AL":"Alabama",
              "AK":"Alaska",
              "AZ":"Arizona",
              "AR":"Arkansas",
              "CA":"California",
              "CO":"Colorado",
              "CT":"Connecticut",
              "DE":"Delaware",
              "DC":"District of Columbia",
              "FL":"Florida",
              "GA":"Georgia",
              "HI":"Hawaii",
              "ID":"Idaho",
              "IL":"Illinois",
              "IN":"Indiana",
              "IA":"Iowa",
              "KS":"Kansas",
              "KY":"Kentucky",
              "LA":"Louisiana",
              "ME":"Maine",
              "MD":"Maryland",
              "MA":"Massachusetts",
              "MI":"Michigan",
              "MN":"Minnesota",
              "MS":"Mississippi",
              "MO":"Missouri",
              "MT":"Montana",
              "NE":"Nebraska",
              "NV":"Nevada",
              "NH":"New Hampshire",
              "NJ":"New Jersey",
              "NM":"New Mexico",
              "NY":"New York",
              "NC":"North Carolina",
              "ND":"North Dakota",
              "MP":"Northern Mariana Islands",
              "OH":"Ohio",
              "OK":"Oklahoma",
              "OR":"Oregon",
              "PA":"Pennsylvania",
              "PR":"Puerto Rico",
              "RI":"Rhode Island",
              "SC":"South Carolina",
              "SD":"South Dakota",
              "TN":"Tennessee",
              "TX":"Texas",
              "UT":"Utah",
              "VT":"Vermont",
              "VA":"Virginia",
              "VI":"Virgin Islands",
              "WA":"Washington",
              "WV":"West Virginia",
              "WI":"Wisconsin",
              "WY":"Wyoming"}
mortaility_data["STATE"]=mortaility_data["STATE"].replace(XX_to_full)
mortaility_data = mortaility_data.rename(columns={"STATE":"Location"})
mortaility_data.head()

,YEAR,Location,RATE,DEATHS
0,2019,Alabama,219.6,13448
1,2019,Alaska,129.7,843
2,2019,Arizona,134.0,12587
3,2019,Arkansas,226.5,8669
4,2019,California,136.9,62394


In [7]:
full_combined =  pd.merge( merged_per_state, mortaility_data, how="left", on=["Location", "Location"])
#full_combined = full_combined.dropna (axis=0,how='any',subset=["Health Spending per Capita"])
full_combined

,Location,Health Spending per Capita,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total,Footnotes,YEAR,RATE,DEATHS
0,United States,"$9,671",0.496,0.059,0.198,0.142,0.014,0.092,1.0,1.0,NaN,NaN,NaN
1,Alabama,"$8,741",0.472,0.055,0.195,0.160,0.021,0.097,1.0,NaN,2019.0,219.6,13448
2,Alaska,"$13,226",0.484,0.035,0.213,0.100,0.053,0.115,1.0,NaN,2019.0,129.7,843
3,Arizona,"$8,145",0.451,0.052,0.210,0.161,0.015,0.111,1.0,NaN,2019.0,134.0,12587
4,Arkansas,"$8,853",0.420,0.054,0.262,0.159,0.014,0.091,1.0,NaN,2019.0,226.5,8669
5,California,"$9,628",0.480,0.066,0.253,0.114,0.009,0.078,1.0,NaN,2019.0,136.9,62394
6,Colorado,"$8,286",0.534,0.069,0.168,0.128,0.023,0.078,1.0,NaN,2019.0,127.7,7762
7,Connecticut,"$11,831",0.529,0.048,0.215,0.141,0.007,0.059,1.0,NaN,2019.0,143.1,7354
8,Delaware,"$12,213",0.497,0.041,0.204,0.173,0.018,0.066,1.0,NaN,2019.0,154.3,2053
9,District of Columbia,"$13,934",0.549,0.065,0.255,0.082,0.013,0.036,1.0,NaN,NaN,NaN,NaN
